In [1]:
import pandas as pd

# 读取CSV文件
file_path = 'wandb_export_2024-09-19T19_18_42.011+08_00.csv'  # 替换为实际文件路径
df = pd.read_csv(file_path)
df = df.drop_duplicates(subset='user_id')

# 查看总行数
total_rows = len(df)
print(f"文件总行数: {total_rows}")

filtered_df = df[df['status'] == 'failed']

# 查看筛选后的行
print(filtered_df)


文件总行数: 2996
     user_id  status assistant_json
444     4798  failed            NaN


In [2]:
# 去掉status为failed的行
df = df[df['status'] != 'failed']

In [3]:
# 筛选出 userid 在 3000 到 5999 范围内的数据
original_userids = set(range(3000, 6000))
unique_userids = set(df['user_id'].unique())

# 找到去重后消失的 userid
removed_userids = original_userids - unique_userids

print("缺少的 userid:")
print(removed_userids)

缺少的 userid:
{5224, 5101, 5198, 5241, 4798}


In [4]:
fix_df = pd.read_csv('wandb_export_2024-09-20T02_09_57.075+08_00.csv')  # 替换为你的修正文件路径

# 获取需要替换的user_id
# replace_user_ids = fix_df['user_id'].unique()
replace_user_ids = [5224, 5101, 5198, 5241, 4798, 3706, 3751, 3797, 3944, 5426, 5585, 5659, 5976]
# 从原始df中删除需要替换的user_id数据
df = df[~df['user_id'].isin(replace_user_ids)]
print(len(df))
print("未命中：")
print(set(replace_user_ids) - set(fix_df['user_id'].unique()))
fix_df = fix_df[fix_df['status'] == 'success']
fix_df = fix_df[fix_df['user_id'].isin(replace_user_ids)]
print("仍然失败：")
print(sorted(set(replace_user_ids) - set(fix_df['user_id'].unique())))

# 将fix_df中的数据合并到df中
df = pd.concat([df, fix_df], ignore_index=True)

2987
未命中：
set()
仍然失败：
[]


In [5]:
# 重新排序（按user_id排序）
df_sorted = df.sort_values(by='user_id').reset_index(drop=True)
# 查看总行数
total_rows = len(df_sorted)
print(f"压缩后的文件总行数: {total_rows}")

# 查看前几行
print("前几行内容:")
print(df_sorted.head())

压缩后的文件总行数: 3000
前几行内容:
   user_id   status                                     assistant_json
0     3000  success  {"reason": "The individual's trajectory shows ...
1     3001  success  {"reason": "The individual's trajectory shows ...
2     3002  success  {"reason": "The individual's trajectory shows ...
3     3003  success  {"reason": "The individual's trajectory shows ...
4     3004  success  {"reason": "The individual's trajectory shows ...


In [6]:
import json

def parse_json_column(json_str):
    return json.loads(json_str)

# 解析assistant_json列
df_sorted['parsed_json'] = df_sorted['assistant_json'].apply(parse_json_column)

In [7]:
df_sorted['prediction'] = df_sorted['parsed_json'].apply(lambda x: x['prediction'] if x else None)

# 查看解析后的数据
print("提取后的前几行内容:")
print(df_sorted[['user_id', 'status', 'prediction']].head())

# 保存提取后的结果为新文件
total_rows = len(df_sorted)
print(f"压缩后的文件总行数: {total_rows}")

提取后的前几行内容:
   user_id   status                                         prediction
0     3000  success  [[60, 13, 196, 20], [60, 14, 196, 19], [60, 15...
1     3001  success  [[60, 14, 114, 115], [60, 15, 117, 117], [60, ...
2     3002  success  [[60, 0, 189, 195], [60, 1, 189, 195], [60, 2,...
3     3003  success  [[60, 1, 182, 28], [60, 2, 177, 29], [60, 8, 1...
4     3004  success  [[60, 1, 121, 116], [60, 2, 120, 116], [60, 3,...
压缩后的文件总行数: 3000


In [8]:
expanded_data = []

# 遍历每一行
for index, row in df_sorted.iterrows():
    user_id = row['user_id']
    predictions = row['prediction']
    
    # 遍历每个预测值，将其添加到expanded_data列表中
    for pred in predictions:  # 使用eval解析字符串形式的列表
        expanded_data.append([user_id] + pred)

# 将展开的数据转换为DataFrame
expanded_df = pd.DataFrame(expanded_data, columns=['user_id', 'd', 't', 'x', 'y'])

# 查看转换后的数据
print(expanded_df.head())
total_rows = len(expanded_df)
print(f"文件总行数: {total_rows}")
# 确保 expanded_df 已经创建
unique_users = expanded_df['user_id'].nunique()
print(f"展开后的数据包含 {unique_users} 个不同的用户。")
expanded_df.to_csv('cityD.csv.gz', compression='gzip', index=False, header=False)

   user_id   d   t    x   y
0     3000  60  13  196  20
1     3000  60  14  196  19
2     3000  60  15  196  20
3     3000  60  16  196  20
4     3000  60  17  196  20
文件总行数: 824733
展开后的数据包含 3000 个不同的用户。
